In [1]:
from ultralytics import YOLO
model = YOLO('yolov8x.pt')

result = model.train(data='dataset.yaml', epochs=300, imgsz=1024)


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


New https://pypi.org/project/ultralytics/8.0.236 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.235 🚀 Python-3.10.13 torch-1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=dataset.yaml, epochs=300, time=None, patience=50, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False,

train: Scanning /data/ephemeral/home/dataset/train.cache... 4000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /data/ephemeral/home/dataset/val.cache... 883 images, 0 backgrounds, 0 corrupt: 100%|██████████| 883/883 [00:00<?, ?it/s]

val: WARNING ⚠️ /data/ephemeral/home/dataset/val/4041.jpg: 1 duplicate labels removed


Plotting labels to /data/ephemeral/home/ultralytics/runs/detect/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/300      31.2G      1.028      3.678      1.556        116       1024:  29%|██▉       | 73/250 [01:10<02:49,  1.04it/s]

In [6]:
import cv2
import torch
import json
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch

# exp_name 수정하여 weights Load
exp_name = 'exp2'

model = torch.hub.load('.', 'custom', path= f'./ultralytics/runs/detect/train2/weights/best.pt', source='local')
# model.conf = 0.001  # confidence threshold (0-1)
# model.iou = 0.6  # NMS IoU threshold (0-1)

prediction_string = ['']  * 4871
image_id = [f'test/{i:04}.jpg' for i in range(4871)]
for i in tqdm(range(4871)):
    img = Image.open(f'./data/images/{i:04}.jpg')

    results = model(img, size=1024, augment=True)
    for bbox in results.pandas().xyxy[0].values:
        xmin, ymin, xmax, ymax, confidence, cls, name = bbox
        prediction_string[i] += f'{cls} {confidence} {xmin} {ymin} {xmax} {ymax} '

raw_data ={
    'PredictionString' : prediction_string,
    'image_id' : image_id
}

dataframe = pd.DataFrame(raw_data)

# csv 형태로 저장
dataframe.to_csv(f'./runs/val/submission_tta_cleaned_{exp_name}.csv', na_rep='NaN',index=None)

FileNotFoundError: [Errno 2] No such file or directory: '/data/ephemeral/home/ultralytics/./hubconf.py'